In [1]:
from kaggle_secrets import UserSecretsClient
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_KEY")

from huggingface_hub import login

login(token=secret_value_0)

2025-05-13 20:36:00.115146: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747168560.314634     474 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747168560.370077     474 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import nltk # to convert text into some more presentable way
nltk.download("punkt")
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
file_name = "/kaggle/input/church-s/church_text"
with open(file_name, "r", encoding="utf-8") as file: ## reading a file and storing it into one variable
    f = file.read()
    sentenses = sent_tokenize(f) # our english teacher
tokenized_sentenses = [word_tokenize(sent) for sent in sentenses] # structure a text

In [4]:
mistral = "mistralai/Mistral-7B-Instruct-v0.3"
mistral_model = AutoModelForCausalLM.from_pretrained(
    mistral,
    torch_dtype=torch.float16, ## gpu support
    device_map=None,
    )
mistral_tokenizer = AutoTokenizer.from_pretrained(mistral)
mistral_tokenizer.pad_token = mistral_tokenizer.eos_token


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
from datasets import Dataset

dataset = Dataset.from_dict({"text": f})

def token_func(example):
    return mistral_tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

token_dataset = dataset.map(token_func, batched=True)

Map:   0%|          | 0/14270 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=mistral_tokenizer,
    mlm=False 
)

training_args = TrainingArguments(
    output_dir="./mistral-continued-pretrain",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=1,
    save_steps=500,
    save_total_limit=1,
    prediction_loss_only=True,
    fp16=True,
    logging_steps=50,
)

trainer = Trainer(
    model=mistral_model,
    args=training_args,
    train_dataset=token_dataset,
    tokenizer=mistral_tokenizer,
    data_collator=data_collator,
)

trainer.train()

/tmp/ipykernel_474/1571323807.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=mistral_model, tokenizer=mistral_tokenizer)
pipe("hello there")